In [8]:
import os 
from glob import glob
import pickle
from Semanticseg import read_image, semantic_seg
import tensorflow as tf

In [15]:
#inputs:
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'

with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    model = data['model']
    classNames = data['classNames']
    tile_size = data['sxy']

tile_size = 1024
# Paths to training and validation datasets:
pthDataset = r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\validation'

# Get paths to validation images and labels
test_images_path = sorted(glob(os.path.join(pthDataset, 'im', "*.png")))
test_masks_path = sorted(glob(os.path.join(pthDataset, 'label','corrected_labels', "*.png")))

In [7]:
y_true = []
y_pred = []

In [11]:
# Read images and masks, perform segmentation, and collect predictions and true labels
for img_path, mask_path in zip(test_images_path, test_masks_path):
    image_tensor = read_image(img_path, tile_size)
    mask_tensor = read_image(mask_path, tile_size, mask=True)

    if image_tensor is not None and mask_tensor is not None:
        print('hello')
        pred_mask = semantic_seg(img_path, tile_size, model)
        true_mask = tf.squeeze(mask_tensor).numpy().astype(int)

        y_true.extend(true_mask.flatten())
        y_pred.extend(pred_mask.flatten())
    else:
        print(f"Error processing image or mask at {img_path} or {mask_path}")
